In [ ]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 38.1 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
import cv2
import tensorflow as tf
from tqdm import tqdm 

from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

from tensorflow.keras.layers.experimental.preprocessing import RandomFlip
import tensorflow.keras.layers as tfl
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving images.zip to images.zip
Saving labels.csv to labels.csv
Saving test_new.csv to test_new.csv
Saving train_new.csv to train_new.csv


In [ ]:
headers = [*pd.read_csv('train_new.csv', nrows=1)]
train_data = pd.read_csv('train_new.csv', usecols=[c for c in headers if c!= 'Unnamed: 0'])

In [ ]:
headers = [*pd.read_csv('test_new.csv', nrows=1)]
test_data = pd.read_csv('test_new.csv', usecols=[c for c in headers if c!= 'Unnamed: 0'])

In [ ]:
labels = pd.read_csv('labels.csv')

In [ ]:
!unzip images.zip

Archive:  images.zip
   creating: images/images/
  inflating: images/images/img0.jpg  
  inflating: images/images/img1.jpg  
  inflating: images/images/img10.jpg  
  inflating: images/images/img100.jpg  
  inflating: images/images/img101.jpg  
  inflating: images/images/img102.jpg  
  inflating: images/images/img103.jpg  
  inflating: images/images/img104.jpg  
  inflating: images/images/img105.jpg  
  inflating: images/images/img106.jpg  
  inflating: images/images/img107.jpg  
  inflating: images/images/img108.jpg  
  inflating: images/images/img109.jpg  
  inflating: images/images/img11.jpg  
  inflating: images/images/img110.jpg  
  inflating: images/images/img111.jpg  
  inflating: images/images/img112.jpg  
  inflating: images/images/img113.jpg  
  inflating: images/images/img114.jpg  
  inflating: images/images/img115.jpg  
  inflating: images/images/img116.jpg  
  inflating: images/images/img117.jpg  
  inflating: images/images/img118.jpg  
  inflating: images/images/img119.jpg

In [ ]:
image_directory = "/content/images/images/"

In [ ]:
train_data

,image_id,l0,l1,l10,l11,l12,l13,l14,l15,l16,...,l83,l84,l85,l86,l87,l88,l89,l9,l90,l91
0,img256.jpg,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,img98.jpg,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,img147.jpg,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,img254.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,img160.jpg,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,img105.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197,img198.jpg,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
198,img82.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
199,img124.jpg,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_data

,image_id
0,img102.jpg
1,img103.jpg
2,img11.jpg
3,img113.jpg
4,img114.jpg
...,...
83,img81.jpg
84,img84.jpg
85,img90.jpg
86,img93.jpg


In [ ]:
labels

,label_id,object
0,l0,people
1,l1,trees
2,l2,grass
3,l3,building
4,l4,snow
...,...,...
87,l87,stage
88,l88,book
89,l89,books
90,l90,flag


In [ ]:
label_id = labels['label_id'].tolist()
label_id

['l0',
 'l1',
 'l2',
 'l3',
 'l4',
 'l5',
 'l6',
 'l7',
 'l8',
 'l9',
 'l10',
 'l11',
 'l12',
 'l13',
 'l14',
 'l15',
 'l16',
 'l17',
 'l18',
 'l19',
 'l20',
 'l21',
 'l22',
 'l23',
 'l24',
 'l25',
 'l26',
 'l27',
 'l28',
 'l29',
 'l30',
 'l31',
 'l32',
 'l33',
 'l34',
 'l35',
 'l36',
 'l37',
 'l38',
 'l39',
 'l40',
 'l41',
 'l42',
 'l43',
 'l44',
 'l45',
 'l46',
 'l47',
 'l48',
 'l49',
 'l50',
 'l51',
 'l52',
 'l53',
 'l54',
 'l55',
 'l56',
 'l57',
 'l58',
 'l59',
 'l60',
 'l61',
 'l62',
 'l63',
 'l64',
 'l65',
 'l66',
 'l67',
 'l68',
 'l69',
 'l70',
 'l71',
 'l72',
 'l73',
 'l74',
 'l75',
 'l76',
 'l77',
 'l78',
 'l79',
 'l80',
 'l81',
 'l82',
 'l83',
 'l84',
 'l85',
 'l86',
 'l87',
 'l88',
 'l89',
 'l90',
 'l91']

In [ ]:
train, val = train_test_split(train_data, test_size=0.1, random_state=111)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    horizontal_flip=True,
    # vertical_flip=True,
    shear_range=0.3,
    zoom_range=0.2,
    rotation_range=15,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2]
)

val_datagen = ImageDataGenerator(rescale = 1./255.)

train_generator = train_datagen.flow_from_dataframe(
    dataframe = train,
    directory = image_directory,
    x_col = "image_id",
    y_col = label_id,
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = "raw",
    image_size = (224, 224),
    target_size = (224, 224)
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe = val,
    directory = image_directory,
    x_col = "image_id",
    y_col = label_id,
    batch_size = 16,
    seed = 42,
    class_mode = "raw",
    image_size = (224, 224),
    target_size = (224, 224)
)

Found 180 validated image filenames.
Found 21 validated image filenames.


In [ ]:
pre_trained_model = tf.keras.applications.DenseNet201(
    input_shape=(224, 224, 3),
    include_top = False,
    weights='imagenet'
)

print(f"Number of layers in the Model: {len(pre_trained_model.layers)}")



74836368/74836368 [==============================] - 4s 0us/step
Number of layers in the Model: 707


In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = pre_trained_model(inputs)
x = tfl.GlobalAveragePooling2D()(x)
x = tfl.Dropout(0.25)(x)
outputs = tfl.Dense(
    len(label_id),
    activation = 'sigmoid'
)(x)

model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 densenet201 (Functional)    (None, 7, 7, 1920)        18321984  
                                                                 
 global_average_pooling2d_1   (None, 1920)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 1920)              0         
                                                                 
 dense_1 (Dense)             (None, 92)                176732    
                                                                 
Total params: 18,498,716
Trainable params: 18,269,660
Non-trainable params: 229,056
_________________________________________

In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss = 'binary_crossentropy',
    metrics = [tf.keras.metrics.BinaryAccuracy(
        name="binary_accuracy",
        dtype=None,
        threshold = 0.5)]
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    patience = 10,
    monitor = 'val_loss',
    restore_best_weights = True
)

history_tuned = model.fit(
    train_generator,
    validation_data = val_generator,
    epochs = 300,
    callbacks=[early_stopping]    
)


Epoch 1/300
6/6 [==============================] - 28s 1s/step - loss: 0.7568 - binary_accuracy: 0.5671 - val_loss: 0.6196 - val_binary_accuracy: 0.6527
Epoch 2/300
6/6 [==============================] - 6s 889ms/step - loss: 0.5996 - binary_accuracy: 0.6745 - val_loss: 0.5287 - val_binary_accuracy: 0.7593
Epoch 3/300
6/6 [==============================] - 5s 797ms/step - loss: 0.4707 - binary_accuracy: 0.7852 - val_loss: 0.4514 - val_binary_accuracy: 0.8452
Epoch 4/300
6/6 [==============================] - 5s 744ms/step - loss: 0.3823 - binary_accuracy: 0.8533 - val_loss: 0.3913 - val_binary_accuracy: 0.8939
Epoch 5/300
6/6 [==============================] - 5s 752ms/step - loss: 0.3042 - binary_accuracy: 0.9118 - val_loss: 0.3459 - val_binary_accuracy: 0.9187
Epoch 6/300
6/6 [==============================] - 5s 747ms/step - loss: 0.2483 - binary_accuracy: 0.9414 - val_loss: 0.3120 - val_binary_accuracy: 0.9327
Epoch 7/300
6/6 [==============================] - 5s 744ms/step - loss:

In [ ]:
val

,image_id,l0,l1,l10,l11,l12,l13,l14,l15,l16,...,l83,l84,l85,l86,l87,l88,l89,l9,l90,l91
23,img154.jpg,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
164,img107.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,img80.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
176,img253.jpg,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
35,img246.jpg,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150,img62.jpg,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57,img56.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81,img78.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39,img200.jpg,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79,img13.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_data

,image_id
0,img102.jpg
1,img103.jpg
2,img11.jpg
3,img113.jpg
4,img114.jpg
...,...
83,img81.jpg
84,img84.jpg
85,img90.jpg
86,img93.jpg


In [ ]:
from tensorflow.keras.utils import img_to_array
image_name = image_directory + 'img102.jpg'
img = image.load_img(image_name, target_size = (224, 224))
img = img_to_array(img)
img = img/255.

print(img.shape)


(224, 224, 3)


In [ ]:
for i, id_image in tqdm(enumerate(test_data['image_id'])):
  print(i, id_image)

88it [00:00, 1701.93it/s]

0 img102.jpg
1 img103.jpg
2 img11.jpg
3 img113.jpg
4 img114.jpg
5 img121.jpg
6 img126.jpg
7 img131.jpg
8 img136.jpg
9 img137.jpg
10 img139.jpg
11 img140.jpg
12 img15.jpg
13 img150.jpg
14 img151.jpg
15 img152.jpg
16 img156.jpg
17 img161.jpg
18 img166.jpg
19 img167.jpg
20 img170.jpg
21 img173.jpg
22 img176.jpg
23 img18.jpg
24 img180.jpg
25 img181.jpg
26 img182.jpg
27 img183.jpg
28 img189.jpg
29 img192.jpg
30 img194.jpg
31 img195.jpg
32 img206.jpg
33 img208.jpg
34 img209.jpg
35 img211.jpg
36 img212.jpg
37 img218.jpg
38 img22.jpg
39 img221.jpg
40 img228.jpg
41 img230.jpg
42 img231.jpg
43 img234.jpg
44 img238.jpg
45 img243.jpg
46 img247.jpg
47 img248.jpg
48 img249.jpg
49 img250.jpg
50 img251.jpg
51 img252.jpg
52 img257.jpg
53 img259.jpg
54 img262.jpg
55 img265.jpg
56 img266.jpg
57 img279.jpg
58 img280.jpg
59 img281.jpg
60 img118.jpg
61 img128.jpg
62 img292.jpg
63 img296.jpg
64 img3.jpg
65 img300.jpg
66 img33.jpg
67 img37.jpg
68 img39.jpg
69 img41.jpg
70 img42.jpg
71 img43.jpg
72 img48.jpg
7

In [ ]:
from tensorflow.keras.utils import img_to_array
prediction = []

for i, id_image in tqdm(enumerate(test_data['image_id'])):
  img_path = image_directory
  img_name = img_path + id_image

  img = image.load_img(img_name, target_size=(224,224))
  img = img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = img/255.
  # print(img.shape)
  # img = img[np.newaxis, ...]
  # print(img.shape)

  pred = model.predict(img)
  # print(pred)
  result = np.where(pred[0] > 0.1, 1, 0)

  res = ''
  for idx, r in enumerate(result):
    if (int(r) == 1):
      res = res + " " + label_id[idx]
      res = res.strip()  
  prediction.append(res)
  print(img_name)

  # try:
  #   img = image.load_img(
  #       img_name,
  #       target_size = (224, 224))
  #   img = img.img_to_array(img)
  #   img = np.expand_dims(img, axis=0)
  #   img = img/255.
  #   # img = img[np.newaxis, ...]

  #   pred = model.predict(img)

  #   result = np.where(pred[0] > 0.45, 1, 0)

  #   res = ''
  #   for idx, r in enumerate(result):
  #     if (int(r) == 1):
  #       res = res + " " + label_id[idx]
  #       res = res.strip()
  #   prediction.append(res)
  
  # except:
  #   prediction.append('')
  #   print(img_name)

0it [00:00, ?it/s]

1/1 [==============================] - 3s 3s/step


1it [00:02,  2.84s/it]

/content/images/images/img102.jpg
1/1 [==============================] - 0s 36ms/step


2it [00:02,  1.24s/it]

...

/content/images/images/img84.jpg
1/1 [==============================] - 0s 38ms/step


86it [00:12,  9.01it/s]

/content/images/images/img90.jpg
1/1 [==============================] - 0s 39ms/step
/content/images/images/img93.jpg
1/1 [==============================] - 0s 39ms/step


88it [00:12,  7.15it/s]

/content/images/images/img94.jpg


In [ ]:
prediction

['l0 l1 l5 l24 l26 l39 l41',
 'l0 l1 l2 l3 l6 l10 l15 l20',
 'l0 l1 l4 l5 l13',
 'l1 l6 l20 l47',
 'l0 l1 l3',
 'l0 l1 l2 l6 l19 l22 l31 l34',
 'l0 l1 l2 l3 l6 l54',
 'l1 l3',
 'l0',
 'l0 l1 l2 l3 l4 l5 l9 l18 l59',
 'l1 l2 l4 l9 l18',
 'l0 l1 l6 l19 l22 l31 l34 l54 l74',
 'l1 l2 l3 l12',
 'l5 l14',
 'l0 l1 l3 l5 l7 l9 l14 l15',
 'l0 l3 l5 l7 l14',
 'l0 l1 l5 l6 l7 l14 l19 l32',
 'l0 l1 l2 l3 l6 l10 l20',
 'l0 l1 l2 l3 l10 l11 l12 l36 l70',
 'l0 l1 l2 l6 l20',
 'l0 l1 l2 l3 l19 l21 l83',
 'l0 l1 l4 l43 l82',
 'l0 l1 l4 l5 l17 l85',
 'l0 l1 l2 l3 l4 l15 l20',
 'l0 l1 l3 l5 l30',
 'l0 l1 l2 l10 l13 l21',
 'l0 l1 l3 l22 l30 l31',
 'l1 l2 l3 l7 l20',
 'l0 l1 l6 l19 l20 l32',
 'l0 l1 l2 l3 l5 l7 l11 l12 l30 l31 l33',
 'l0 l1 l2 l3 l5 l6 l11 l22 l30 l31',
 'l0 l1 l2 l3 l5 l6',
 'l0 l1 l2 l5',
 'l1 l2',
 'l1 l2 l5 l10 l13 l32 l36 l70',
 'l0 l1 l2 l3 l5 l7 l15 l18',
 'l0 l1 l2 l3 l5 l6 l20',
 'l8 l30 l70 l78 l88',
 'l1 l2 l18 l21 l71',
 'l8 l13 l23 l66',
 'l0 l1 l3 l4 l17',
 'l0 l1 l3 l4 l5 l9

In [ ]:
test_submission = pd.DataFrame()
test_submission['image_id'] = test_data['image_id']
test_submission['labels'] = prediction

In [ ]:
test_submission

,image_id,labels
0,img102.jpg,l0 l1 l5 l24 l26 l39 l41
1,img103.jpg,l0 l1 l2 l3 l6 l10 l15 l20
2,img11.jpg,l0 l1 l4 l5 l13
3,img113.jpg,l1 l6 l20 l47
4,img114.jpg,l0 l1 l3
...,...,...
83,img81.jpg,l1 l2 l8
84,img84.jpg,l8 l13 l23
85,img90.jpg,l0 l26 l62 l90
86,img93.jpg,l25 l29


In [ ]:
test_submission.to_csv('test_submission.csv', index=False)